## Create 16x results

In [1]:
%%bash

# 1. Define the four similarity functions and four probe‐sets:
similarity_fns=(cos_similarity rank_reorder wpmi soft_wpmi)
# d_probes=(cifar100_train broden imagenet_val imagenet_broden)
d_probes=(imagenet_broden)

# 2. Loop over each pair and invoke describe_neurons.py on the FC layer:
for sim_fn in "${similarity_fns[@]}"; do
  for dp in "${d_probes[@]}"; do
    echo "Running: similarity_fn=$sim_fn, d_probe=$dp"
    python describe_neurons.py \
      --clip_model    ViT-B/16 \
      --target_model  resnet50 \
      --target_layers fc \
      --d_probe       "$dp" \
      --concept_set   data/imagenet_labels.txt \
      --pool_mode     avg \
      --similarity_fn "$sim_fn" \
      --batch_size    200 \
      --device        cuda
    sleep 70
  done
done

Running: similarity_fn=cos_similarity, d_probe=imagenet_broden


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /tmp/xdg-cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 235MB/s]
100%|██████████| 1/1 [00:00<00:00,  7.23it/s]


Running: similarity_fn=rank_reorder, d_probe=imagenet_broden


100%|██████████| 1000/1000 [01:13<00:00, 13.51it/s]


Running: similarity_fn=wpmi, d_probe=imagenet_broden


100%|██████████| 1000/1000 [00:00<00:00, 4011.45it/s]


Running: similarity_fn=soft_wpmi, d_probe=imagenet_broden


100%|██████████| 1000/1000 [00:06<00:00, 161.84it/s]


torch.Size([1000, 1000])


## Similarity function comparison

In [1]:
import os
os.chdir("/home/vpalaniappan/private/DSC291-CLIP-Dissect")

import torch
from sentence_transformers import SentenceTransformer
from sklearn import metrics

import clip
import utils
import similarity

2025-06-03 09:15:46.270193: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-03 09:15:46.270217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-03 09:15:46.271414: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-03 09:15:46.277244: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Settings

In [2]:
similarity_fns = ["cos_similarity", "rank_reorder", "wpmi", "soft_wpmi"]
# similarity_fns = ["rank_reorder"]
# d_probes = ['cifar100_train', 'broden', 'imagenet_val', 'imagenet_broden']
d_probes = ['imagenet_broden']

clip_name = 'ViT-B/16'
target_name = 'resnet50'
target_layer = 'fc'
batch_size = 200
device = 'cuda'
pool_mode = 'avg'
save_dir = 'saved_activations'

In [3]:
model = SentenceTransformer('all-mpnet-base-v2')
clip_model, _ = clip.load(clip_name, device=device)

with open("data/imagenet_labels.txt", "r") as f:
    cls_id_to_name = f.read().split("\n")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Cos similarities

In [4]:
import torch
concept_set = 'data/20k.txt'

with open(concept_set, 'r') as f:
    words = f.read().split('\n')

for similarity_fn in similarity_fns:
    for d_probe in d_probes:
        utils.save_activations(clip_name = clip_name, target_name = target_name, target_layers = [target_layer], 
                               d_probe = d_probe, concept_set = concept_set, batch_size = batch_size, 
                               device = device, pool_mode=pool_mode, save_dir = save_dir)

        save_names = utils.get_save_names(clip_name = clip_name, target_name = target_name,
                                          target_layer = target_layer, d_probe = d_probe,
                                          concept_set = concept_set, pool_mode=pool_mode,
                                          save_dir = save_dir)

        target_save_name, clip_save_name, text_save_name = save_names

        torch.cuda.empty_cache()

        similarities, target_feats = utils.get_similarity_from_activations(target_save_name, clip_save_name, 
                                                                           text_save_name, 
                                                                           eval("similarity.{}".format(similarity_fn)),
                                                                           device=device)

        clip_preds = torch.argmax(similarities, dim=1)
        clip_preds = [words[int(pred)] for pred in clip_preds]

        clip_cos, mpnet_cos = utils.get_cos_similarity(clip_preds, cls_id_to_name, clip_model, model, device, batch_size)
        print("Similarity fn: {}, D_probe: {}".format(similarity_fn, d_probe))
        print("Clip similarity: {:.4f}, mpnet similarity: {:.4f}".format(clip_cos, mpnet_cos))

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /tmp/xdg-cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 328MB/s]
100%|██████████| 1/1 [00:01<00:00,  1.15s/it]


Similarity fn: cos_similarity, D_probe: imagenet_broden
Clip similarity: 0.6470, mpnet similarity: 0.2592


100%|██████████| 1000/1000 [04:43<00:00,  3.53it/s]


Similarity fn: rank_reorder, D_probe: imagenet_broden
Clip similarity: 0.7866, mpnet similarity: 0.5042


100%|██████████| 1000/1000 [00:39<00:00, 25.25it/s]


Similarity fn: wpmi, D_probe: imagenet_broden
Clip similarity: 0.7930, mpnet similarity: 0.5264


100%|██████████| 1000/1000 [00:42<00:00, 23.48it/s]


torch.Size([1000, 20000])
Similarity fn: soft_wpmi, D_probe: imagenet_broden
Clip similarity: 0.7900, mpnet similarity: 0.5242


# Accuracies

In [5]:
def get_topk_acc(sim, k=5):
    correct = 0
    for orig_id in range(1000):
        vals, ids = torch.topk(sim[orig_id], k=k)
        for idx in ids[:k]:
            correct += (int(idx)==orig_id)
    return (correct/1000)*100

def get_correct_rank_mean_median(sim):
    ranks = []
    for orig_id in range(1000):
        vals, ids = torch.sort(sim[orig_id], descending=True)
        
        ranks.append(list(ids).index(orig_id)+1)
        
    mean = sum(ranks)/len(ranks)
    median = sorted(ranks)[500]
    return mean, median

def get_auc(sim):
    max_sim, preds = torch.max(sim.cpu(), dim=1)
    gtruth = torch.arange(0, 1000)
    correct = (preds==gtruth)
    fpr, tpr, thresholds = metrics.roc_curve(correct, max_sim)
    auc = metrics.roc_auc_score(correct, max_sim)
    return auc

In [6]:
concept_set = 'data/imagenet_labels.txt'
with open(concept_set, 'r') as f: 
    words = (f.read()).split('\n')
    

for similarity_fn in similarity_fns:
    for d_probe in d_probes:
        utils.save_activations(clip_name = clip_name, target_name = target_name, target_layers = [target_layer], 
                               d_probe = d_probe, concept_set = concept_set, batch_size = batch_size, 
                               device = device, pool_mode=pool_mode, save_dir = save_dir)

        save_names = utils.get_save_names(clip_name = clip_name, target_name = target_name,
                                          target_layer = target_layer, d_probe = d_probe,
                                          concept_set = concept_set, pool_mode=pool_mode,
                  
                                          save_dir = save_dir)

        target_save_name, clip_save_name, text_save_name = save_names

        similarities, target_feats = utils.get_similarity_from_activations(target_save_name, clip_save_name, 
                                                                           text_save_name, 
                                                                           eval("similarity.{}".format(similarity_fn)),
                                                                           device=device)
        
        print("Similarity fn: {}, D_probe: {}".format(similarity_fn, d_probe))
        print("Top 1 acc: {:.2f}%, Top 5 acc: {:.2f}%".format(get_topk_acc(similarities, k=1),
                                                         get_topk_acc(similarities, k=5)))
        
        mean, median = get_correct_rank_mean_median(similarities)
        print("Mean rank of correct class: {:.2f}, Median rank of correct class: {}".format(mean, median))
        print("AUC: {:.4f}".format(get_auc(similarities)))



100%|██████████| 1/1 [00:00<00:00, 19.70it/s]


Similarity fn: cos_similarity, D_probe: imagenet_broden
Top 1 acc: 11.20%, Top 5 acc: 34.80%
Mean rank of correct class: 31.39, Median rank of correct class: 12
AUC: 0.4924


100%|██████████| 1000/1000 [00:55<00:00, 17.89it/s]


Similarity fn: rank_reorder, D_probe: imagenet_broden
Top 1 acc: 89.90%, Top 5 acc: 98.30%
Mean rank of correct class: 2.12, Median rank of correct class: 1
AUC: 0.6032


100%|██████████| 1000/1000 [00:00<00:00, 5637.06it/s]


Similarity fn: wpmi, D_probe: imagenet_broden
Top 1 acc: 86.90%, Top 5 acc: 98.00%
Mean rank of correct class: 1.98, Median rank of correct class: 1
AUC: 0.7298


100%|██████████| 1000/1000 [00:36<00:00, 27.55it/s]


torch.Size([1000, 1000])
Similarity fn: soft_wpmi, D_probe: imagenet_broden
Top 1 acc: 95.40%, Top 5 acc: 99.00%
Mean rank of correct class: 1.18, Median rank of correct class: 1
AUC: 0.9173
